#Домашнее задание 3. Группировка данных и оконные функции

### 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [ ]:
select job_industry_category as "Сфера деятельности", count(distinct customer_id) as "Кол-во клиентов"
from dz3.customer c
group by job_industry_category
order by 2 desc

### 2. Найти общую сумму дохода (list_price*quantity) по всем подтвержденным заказам за каждый месяц по сферам деятельности клиентов. Отсортировать результат по году, месяцу и сфере деятельности.::

In [ ]:
select
	extract(year from o.order_date) as year
    ,extract(month from o.order_date) as month
    ,job_industry_category AS "Сфера деятельности"
    ,SUM(DISTINCT list_price * quantity) AS "Общая сумма дохода"
FROM customer c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON oi.order_id = o.order_id
LEFT JOIN product p ON p.product_id = oi.product_id
WHERE order_status = 'Approved'
GROUP by extract(year from o.order_date), extract(month from o.order_date), job_industry_category


### 3. Вывести количество уникальных онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. Включить бренды, у которых нет онлайн-заказов от IT-клиентов, — для них должно быть указано количество 0.

In [ ]:
select
	case
		when brand = '' then 'NoNameBrand' --обрабатываем отсутствие бренда
		else brand
	end,
    COALESCE(COUNT(DISTINCT order_id), 0) AS online_orders_count
FROM product p
LEFT JOIN (
    -- Подзапрос: выбираем только онлайн-заказы IT-клиентов с подтверждённым статусом
    SELECT DISTINCT
        o.order_id,
        oi.product_id
    FROM orders o
    JOIN customer c ON o.customer_id = c.customer_id
    JOIN order_items oi ON oi.order_id = o.order_id
    WHERE o.order_status = 'Approved'
      AND c.job_industry_category = 'IT'
      AND o.online_order = TRUE
) AS it_online_orders ON p.product_id = it_online_orders.product_id
GROUP BY p.brand
ORDER BY online_orders_count DESC;


--В таблицах нет брендов, у которых нет онлайн-заказов от IT-клиентов. По всем брендам есть хоть один онлайн-заказ.

### 4. Найти по всем клиентам: сумму всех заказов (общего дохода), максимум, минимум и количество заказов, а также среднюю сумму заказа по каждому клиенту. Отсортировать результат по убыванию суммы всех заказов и количества заказов. Выполнить двумя способами: используя только GROUP BY и используя только оконные функции. Сравнить результат.

In [ ]:

-- 1 способ используя только GROUP BY
select
	c.customer_id
    ,SUM(distinct list_price * quantity) AS "Общая сумма дохода"
    ,max(distinct list_price * quantity) as "Максимум"
    ,min(distinct list_price * quantity) as "Минимум"
    ,count(o.order_id) as "Количество заказов"
    ,SUM(distinct list_price * quantity) / count(o.order_id) as "Средняя сумма заказа"
FROM customer c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON oi.order_id = o.order_id
LEFT JOIN product p ON p.product_id = oi.product_id
--WHERE order_status = 'Approved'
GROUP by c.customer_id
having count(o.order_id) > 0
order by SUM(distinct list_price * quantity) desc, count(o.order_id) desc

-- 2 споособ используя только оконные функции
SELECT DISTINCT
    c.customer_id,
    oi.order_id,
    p.list_price,
    oi.quantity,
    (p.list_price * oi.quantity) AS "Сумма позиции",
    -- Накопительные оконные функции по клиенту
    SUM(p.list_price * oi.quantity)
        OVER (PARTITION BY c.customer_id) AS "Общая сумма дохода",
    MAX(p.list_price * oi.quantity)
        OVER (PARTITION BY c.customer_id) AS "Максимум",
    MIN(p.list_price * oi.quantity)
        OVER (PARTITION BY c.customer_id) AS "Минимум",
    COUNT(o.order_id)
        OVER (PARTITION BY c.customer_id) AS "Количество заказов",
    AVG(p.list_price * oi.quantity)
        OVER (PARTITION BY c.customer_id) AS "Средняя сумма позиции"
FROM customer c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON oi.order_id = o.order_id
LEFT JOIN product p ON p.product_id = oi.product_id
WHERE o.order_id is not null
ORDER BY
    "Общая сумма дохода" DESC,
    "Количество заказов" DESC;


--Вывод: если нужно агрегировать информацию по всем заказам клиента, то нагляднее информация представлена в 1 способе с group by.
--Если нужна информация по клиенту в разрезе заказов, то лучше использовать оконные функции.


### 5. Найти имена и фамилии клиентов с топ-3 минимальной и топ-3 максимальной суммой транзакций за весь период (учесть клиентов, у которых нет заказов, приняв их сумму транзакций за 0).

In [ ]:
WITH customer_total_spent AS (
    SELECT
        c.customer_id,
        c.first_name,
        c.last_name,
        COALESCE(SUM(oi.item_list_price_at_sale * oi.quantity), 0) AS total_spent
    FROM customer c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    LEFT JOIN order_items oi ON oi.order_id = o.order_id
    GROUP BY c.customer_id, c.first_name, c.last_name
),
ranked_customers AS (
    SELECT
        customer_id,
        first_name,
        last_name,
        total_spent,
        RANK() OVER (ORDER BY total_spent DESC) AS rank_desc,
        RANK() OVER (ORDER BY total_spent ASC) AS rank_asc
    FROM customer_total_spent
)
(SELECT
    first_name,
    last_name,
    total_spent,
    rank_desc
FROM ranked_customers
ORDER BY
    rank_desc asc
limit 3)
union all
(SELECT
    first_name,
    last_name,
    total_spent,
    rank_desc
FROM ranked_customers
ORDER BY
    rank_desc DESC
limit 3)



### 6. Вывести только вторые транзакции клиентов (если они есть) с помощью оконных функций. Если у клиента меньше двух транзакций, он не должен попасть в результат.


In [ ]:
SELECT
    c.first_name,
    c.last_name,
    o.order_id AS second_transaction_id
FROM customer c
INNER JOIN (
    SELECT
        customer_id,
        order_id,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_id) AS rn
    FROM orders
) o ON c.customer_id = o.customer_id
WHERE o.rn = 2
ORDER BY c.customer_id;

### 7. Вывести имена, фамилии и профессии клиентов, а также длительность максимального интервала (в днях) между двумя последовательными заказами. Исключить клиентов, у которых только один или меньше заказов.

In [ ]:
select
	first_name
	,last_name,job_title, max(order_date-next_date)
from (
select
	first_name
	,last_name
	,c.customer_id
	,job_title
	,o.order_date
	,lag(o.order_date) over (partition by c.customer_id ORDER BY o.order_date) as next_date
FROM customer c
LEFT JOIN orders o ON c.customer_id = o.customer_id
order by c.customer_id, order_date
)
where next_date is not null
group by first_name
	,last_name, job_title
order by 4 desc

### 8. Найти топ-5 клиентов (по общему доходу) в каждом сегменте благосостояния (wealth_segment). Вывести имя, фамилию, сегмент и общий доход. Если в сегменте менее 5 клиентов, вывести всех.

In [ ]:

select *
from (
select
    row_number() over (partition by c.wealth_segment order by SUM(DISTINCT list_price * quantity) desc) as "Топ-5 клиентов"
    ,wealth_segment
	,first_name
	,c.last_name
    ,SUM(DISTINCT list_price * quantity) AS "Общий доход"
FROM customer c
LEFT JOIN orders o ON c.customer_id = o.customer_id
LEFT JOIN order_items oi ON oi.order_id = o.order_id
LEFT JOIN product p ON p.product_id = oi.product_id
where quantity is not null
group by first_name
	,c.last_name
	,wealth_segment
	)
where "Топ-5 клиентов" < 6